# Concepts

## DataArray and Dataset slicing

In [ ]:
import scipp as sc

x = sc.Variable(dims=['x'], values=[1,2,3,4])
da = sc.DataArray(data=x,
                 coords={'x':x},
                 masks={'x':sc.less(x, 2 * sc.units.one)})
ds = sc.Dataset({'a':da})


Consider a data array `da` and a dataset `ds` with an aligned coord and an aligned mask.
The following conditions must hold:

In [ ]:
assert 'x' in da['x', 0:1].coords # range slice preserves coord as aligned
assert 'x' in da['x', 0:1].masks # range slice preserves coord as aligned
assert 'x' in da['x', 0].attrs # todo, should be unaligned coord
#assert 'x' in da['x', 0].coords # todo, point slice preserves coord as unaligned
#assert 'x' in da['x', 0].masks # todo, point slice preserves masks as aligned

In [ ]:
assert ds['a']['x', 0:1] == ds['x', 0:1]['a']
assert ds['a']['x', 0] == ds['x', 0]['a']

In [ ]:
assert 'x' in ds['a'].coords
assert 'x' in ds['x', 0:1].coords
assert 'x' not in ds['x', 0].coords # cannot have unaligned coord in dataset
assert 'x' in ds['x', 0:1]['a'].coords
assert 'x' in ds['a']['x', 0].attrs # todo, should be unaligned coord
#assert 'x' in ds['x', 0]['a'].coords # todo

assert 'x' in ds['a'].masks
assert 'x' in ds['x', 0:1].masks
#assert 'x' in ds['x', 0].masks # todo, should stay aligned mask
assert 'x' in ds['x', 0:1]['a'].masks
#assert 'x' in ds['a']['x', 0].masks # todo, should stay aligned mask
#assert 'x' in ds['x', 0]['a'].masks # todo, should stay aligned mask

In operations, aligned coords are compared:

In [ ]:
try:
    ok = da['x', 0:1] + da['x', 1:2]
except:
    ok = False
assert not ok

Mismatching unaligned coords are dropped:

In [ ]:
assert da + da['x', 0] == da + da['x', 0].data

A missing unaligned coord is interpreted as mismatch to ensure that:

In [ ]:
a = da['x', 0]
b = da['x', 1]
c = da['x', 2]
assert a + (b + c) == (a + b) + c

Insertion order does not matter for unaligned coords:

In [ ]:
a = da.copy()
a.attrs['x'] = 1.0 * sc.units.m # todo, should be unaligned coord
b = da.copy()
b.attrs['x'] = 2.0 * sc.units.m # todo, should be unaligned coord
ds1 = sc.Dataset()
ds2 = sc.Dataset()
ds1['a'] = a
ds1['b'] = b
ds2['b'] = b
ds2['a'] = a
assert ds1 == ds2

In [ ]:
edges = sc.Variable(dims=['x'], values=[1,2,3,4,5])
da.coords['x'] = edges
assert sc.concatenate(da['x', :2], da['x', 2:], 'x') == da
#assert sc.concatenate(da['x', 0], da['x', 1], 'x') == da['x', 0:2] # todo, concat of unaligned should give aligned, at least for dimension-coords
#assert sc.concatenate(da['x', :-1], da['x', -1], 'x') == da # todo, attr should be unaligned coord and concate of aligned and unaligned should work

Check slicing of 2D coords:

In [ ]:
import numpy as np

var_y = sc.Variable(['y'], values=[3, 4], unit=sc.units.dimensionless)
var_yx= sc.Variable(['y', 'x'], values=np.arange(10).reshape(2,5), unit=sc.units.angstrom)
data_yx= sc.Variable(['y', 'x'], values=np.flip(np.arange(10.0).reshape(2,5)), unit=sc.units.counts)
da = sc.DataArray(data=data_yx, coords={'x':var_yx,'y':var_y})

da['x', 0] + da['x', 1] # Allowed y-coord same. x-coord differ but not aligned. Correct behaviour
try:
    da['y', 0] + da['y', 1] # Not allowed x-coords aligned and differ. Correct behaviour
except RuntimeError:
    ok = False
else:
    ok = True
assert not ok